<a href="https://colab.research.google.com/github/mett29/DL-Competition/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [75]:
from getpass import getpass
import os

user = getpass('Github user')
password = getpass('Github password')
os.environ['GITHUB_AUTH'] = user + ':' + password

#!git clone https://$GITHUB_AUTH@github.com/mett29/DL-Competition.git
#!git submodule update --init --recursive
!git clone https://$GITHUB_AUTH@github.com/DanieleParravicini/DL-CompetitionsDatasets

Github user··········
Github password··········
Cloning into 'DL-CompetitionsDatasets'...
remote: Enumerating objects: 2079, done.
remote: Total 2079 (delta 0), reused 0 (delta 0), pack-reused 2079
Receiving objects: 100% (2079/2079), 90.16 MiB | 12.41 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [0]:
!apt-get install tree

In [29]:
PATH = "DL-CompetitionsDatasets/Classification_Dataset"
!tree DL-Competition/Classification_Dataset/ -d

DL-Competition/Classification_Dataset/
├── test
└── training
    ├── airplanes
    ├── bear
    ├── calculator
    ├── computer-monitor
    ├── fireworks
    ├── galaxy
    ├── grand-piano
    ├── kangaroo
    ├── laptop
    ├── lightbulb
    ├── lightning
    ├── mountain-bike
    ├── owl
    ├── school-bus
    ├── sheet-music
    ├── skyscraper
    ├── sword
    ├── t-shirt
    ├── waterfall
    └── wine-bottle

22 directories


In [0]:
SEED = 1234
tf.random.set_seed(SEED) 

In [0]:
apply_data_augmentation = False

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split=0.2)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        fill_mode='constant',
                                        cval=0,
                                        validation_split=0.2)

# Create test ImageDataGenerator objects
test_data_gen = ImageDataGenerator(rescale=1./255)

In [68]:
batch_size = 128
no_epochs = 20
no_classes = 20
validation_split = 0.2

IMG_HEIGHT = 256
IMG_WIDTH = 256

decide_class_indices = False
if decide_class_indices:
    classes = ['airplanes',           # 0
               'bear',                # 1
               'calculator',          # 2
               'computer-monitor',    # 3
               'fireworks',           # 4
               'galaxy',              # 5
               'grand-piano',         # 6
               'kangaroo',            # 7
               'laptop',              # 8
               'lightbulb',           # 9
               'lightning',           # 10
               'mountain-bike',       # 11
               'owl',                 # 12
               'school-bus',          # 13
               'sheet-music',         # 14
               'skyscraper',          # 15
               'sword',               # 16
               't-shirt',             # 17
               'waterfall',           # 18
               'wine-bottle']         # 19
else:
    classes=None

dataset_dir = os.path.join(PATH, 'training')

train_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               batch_size=batch_size,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset='training',
                                               seed=SEED)

# Validation
valid_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               batch_size=batch_size, 
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               subset='validation',
                                               seed=SEED)

Found 1247 images belonging to 20 classes.
Found 307 images belonging to 20 classes.


In [0]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_HEIGHT, IMG_WIDTH, 3], [None, no_classes]))

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_HEIGHT, IMG_WIDTH, 3], [None, no_classes]))

valid_dataset = valid_dataset.repeat()

In [70]:
train_gen.class_indices

{'airplanes': 0,
 'bear': 1,
 'calculator': 2,
 'computer-monitor': 3,
 'fireworks': 4,
 'galaxy': 5,
 'grand-piano': 6,
 'kangaroo': 7,
 'laptop': 8,
 'lightbulb': 9,
 'lightning': 10,
 'mountain-bike': 11,
 'owl': 12,
 'school-bus': 13,
 'sheet-music': 14,
 'skyscraper': 15,
 'sword': 16,
 't-shirt': 17,
 'waterfall': 18,
 'wine-bottle': 19}

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(20, activation='softmax')
])

In [72]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 65536)            

In [0]:
# Optimization parameters
loss = tf.keras.losses.CategoricalCrossentropy()
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [74]:
model.fit(x=train_dataset,
          epochs=epochs,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen)
         )

Train for 10 steps, validate for 3 steps
Epoch 1/15
10/10 [==============================] - 16s 2s/step - loss: 4.0702 - accuracy: 0.0690 - val_loss: 2.9515 - val_accuracy: 0.0586
Epoch 2/15
10/10 [==============================] - 9s 940ms/step - loss: 2.7702 - accuracy: 0.1772 - val_loss: 2.6375 - val_accuracy: 0.2606
Epoch 3/15
10/10 [==============================] - 9s 936ms/step - loss: 2.1684 - accuracy: 0.3705 - val_loss: 2.2790 - val_accuracy: 0.3388
Epoch 4/15
10/10 [==============================] - 9s 931ms/step - loss: 1.6259 - accuracy: 0.5309 - val_loss: 2.0793 - val_accuracy: 0.4137
Epoch 5/15
10/10 [==============================] - 9s 928ms/step - loss: 1.2161 - accuracy: 0.6480 - val_loss: 1.9838 - val_accuracy: 0.4397
Epoch 6/15
10/10 [==============================] - 9s 933ms/step - loss: 0.8049 - accuracy: 0.7755 - val_loss: 2.0073 - val_accuracy: 0.4691
Epoch 7/15
10/10 [==============================] - 9s 941ms/step - loss: 0.5150 - accuracy: 0.8653 - val_los